# Create a block from scratch

In [38]:
# the hash library
import hashlib as hash
# data encoding service
import pickle

our block of data will contain many fields such as an:
    identifier
    time
    previous hash
    merkle root
    list of transactions
These can be stored in a python dictionary which is a key-value structure

dict = { key:value,
         key2:value2,
        .
        .
        .
        keyn:valuen   
}

In [39]:
# initialize a block. Note 'transactions' is initialized as an empty list
block = {
    'height':1,
    'time':0,
    'prevHash':'null',
    'merkleRoot': 'abcd',
    'transactions': []
        }
print(block)

{'height': 1, 'time': 0, 'prevHash': 'null', 'merkleRoot': 'abcd', 'transactions': []}


Let's create a transaction to store in our blockchain. Remember a transaction is just data; this can be anything represented as a digital object.

In [40]:
# create a transaction (string)
transaction='Pay $1,000,000 to Jeff'
print(transaction)

Pay $1,000,000 to Jeff


To store the transaction object, we will hash it to create a unique identifier of the information

In [41]:
hashed_tx = hash.sha1(transaction)
print(hashed_tx)

TypeError: Unicode-objects must be encoded before hashing

```p
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-34-6b646aa88a56> in <module>
----> 1 hashed_tx = hash.sha1(transaction)
      2 print(hashed_tx)

TypeError: Unicode-objects must be encoded before hashing
```

The error message above is telling us that we cannot hash a string object such as 'Pay $1,000,000 to Jeff'

In [42]:
encoded_tx = transaction.encode()
print(encoded_tx)

b'Pay $1,000,000 to Jeff'


the 'b' is telling us that the string is now a byte object